In [1]:
import os
print(os.getcwd())
os.makedirs("MLflowLogs", exist_ok=True)

/Users/sebaalobaidani


In [4]:
# Show unique neighborhoods
print("\nNeighborhoods:")
print(df['sna_neighborhood'].dropna().unique())

# Show some sample crime types
print("\nOffense Types:")
print(df['incident_type_desc'].dropna().unique()[:20])

# Check if you have August 2022 and July 2023
print("\nDates available in data:")
print(df['create_time_incident'].dropna().dt.to_period('M').unique())



Neighborhoods:
['MT. AIRY' 'WESTWOOD' 'WEST END' 'SOUTH FAIRMOUNT' 'CAMP WASHINGTON'
 'CORRYVILLE' 'EAST END' 'DOWNTOWN' 'AVONDALE' 'OVER-THE-RHINE'
 'WEST PRICE HILL' 'PENDLETON' 'CUF' 'NORTH FAIRMOUNT' 'EAST PRICE HILL'
 'CLIFTON' 'OAKLEY' 'MT. AUBURN' 'EAST WESTWOOD' 'PLEASANT RIDGE'
 'VILLAGES AT ROLL HILL' 'MADISONVILLE' 'SPRING GROVE VILLAGE' 'HARTWELL'
 'MT. WASHINGTON' 'BOND HILL' 'WALNUT HILLS' 'NORTHSIDE' 'ROSELAWN'
 'WINTON HILLS' 'QUEENSGATE' 'LOWER PRICE HILL' 'HYDE PARK' 'CARTHAGE'
 'NORTH AVONDALE - PADDOCK HILLS' 'KENNEDY HEIGHTS' 'COLLEGE HILL'
 'SOUTH CUMMINSVILLE' 'EVANSTON' 'RIVERSIDE' 'EAST WALNUT HILLS'
 'MT. ADAMS' 'LINWOOD' 'MT. LOOKOUT' 'MILLVALE' 'SAYLER PARK'
 'SEDAMSVILLE' 'ENGLISH WOODS' 'CALIFORNIA' 'COLUMBIA TUSCULUM']

Offense Types:
['MENTALLY IMPAIRED NON VIOL' 'ADVISED INCIDENT' 'PRISONER PICK UP'
 'DIRECTED PATROL - VEHICLE' 'TOWED VEH - FOR RECORDS USE ONLY'
 'DISORDERLY PERSON(S)' 'NOISE COMPLAINT' 'FAMILY TROUBLE - NON VIOLENT'
 '911 CALL NO EMER

In [15]:
# Force load directly from your Desktop
df = pd.read_csv("/Users/sebaalobaidani/Desktop/calls_for_service_latest.csv", low_memory=False)

# Check the column names
print("✅ Loaded file successfully")
print("Number of rows:", len(df))
print("Column names:", df.columns.tolist())

# Peek at the first 5 values from the date column
print("\n🕓 Sample values from the date column:")
for col in df.columns:
    if "date" in col.lower() or "time" in col.lower():
        print(f"\n→ Column: {col}")
        print(df[col].head(5))

✅ Loaded file successfully
Number of rows: 5894564
Column names: ['ADDRESS_X', 'AGENCY', 'CREATE_TIME_INCIDENT', 'DISPOSITION_TEXT', 'EVENT_NUMBER', 'INCIDENT_TYPE_ID', 'INCIDENT_TYPE_DESC', 'PRIORITY', 'PRIORITY_COLOR', 'ARRIVAL_TIME_PRIMARY_UNIT', 'CLOSED_TIME_INCIDENT', 'DISPATCH_TIME_PRIMARY_UNIT', 'BEAT', 'DISTRICT', 'SNA_NEIGHBORHOOD', 'CPD_NEIGHBORHOOD', 'COMMUNITY_COUNCIL_NEIGHBORHOOD', 'LATITUDE_X', 'LONGITUDE_X']

🕓 Sample values from the date column:

→ Column: CREATE_TIME_INCIDENT
0    11/15/2021 11:03:58 PM
1    11/15/2021 11:03:34 PM
2    11/15/2021 11:00:58 PM
3    11/15/2021 11:00:01 PM
4    11/15/2021 10:59:48 PM
Name: CREATE_TIME_INCIDENT, dtype: object

→ Column: ARRIVAL_TIME_PRIMARY_UNIT
0    11/15/2021 11:13:48 PM
1                       NaN
2                       NaN
3    11/15/2021 11:28:55 PM
4                       NaN
Name: ARRIVAL_TIME_PRIMARY_UNIT, dtype: object

→ Column: CLOSED_TIME_INCIDENT
0    11/15/2021 11:46:53 PM
1    11/15/2021 11:03:34 PM
2    11/

In [3]:
# FINAL CODE

# --- Imports ---
import pandas as pd
import mlflow
import transformers
import huggingface_hub
import time
from transformers import pipeline
import warnings
import os
import calendar
import re
warnings.simplefilter(action='ignore', category=pd.errors.DtypeWarning)

# Set up MLflow
os.makedirs("/Users/sebaalobaidani/Desktop/MLflowLogs", exist_ok=True)
mlflow.set_tracking_uri("file:///Users/sebaalobaidani/Desktop/MLflowLogs")
mlflow.set_experiment("Cincinnati Crime Chatbot")

# --- Load CSV and format date ---
def load_crime_data():
    df = pd.read_csv("/Users/sebaalobaidani/Desktop/calls_for_service_latest.csv", low_memory=False)
    df.columns = [col.lower() for col in df.columns]

    if 'create_time_incident' in df.columns:
        df['create_time_incident'] = pd.to_datetime(df['create_time_incident'], 
                                                    format="%m/%d/%Y %I:%M:%S %p", 
                                                    errors='coerce')

    print("\n✅ Sample Neighborhoods:")
    print(df['sna_neighborhood'].dropna().unique()[:10])
    print("\n📅 Sample Dates (Month-Year):")
    print(df['create_time_incident'].dropna().dt.to_period('M').unique()[:10])
    return df

# --- Filter based on question ---
def get_relevant_rows(question, df):
    q = question.lower()
    filtered = df.copy()

    neighborhood_match = None
    offense_match = None
    month_match = None
    year_match = None

    # --- Neighborhood match ---
    if 'sna_neighborhood' in df.columns:
        for neighborhood in df['sna_neighborhood'].dropna().unique():
            if neighborhood.lower() in q:
                neighborhood_match = neighborhood
                break
    print(f"📍 Neighborhood match: {neighborhood_match}")

    if neighborhood_match is None:
        return pd.DataFrame(columns=df.columns)

    # --- Offense match ---
    if 'incident_type_desc' in df.columns:
        for offense in df['incident_type_desc'].dropna().unique():
            if offense.lower() in q:
                offense_match = offense
                break
    print(f"🚨 Offense match: {offense_match}")

    # --- Month and Year match ---
    months = {month.lower(): idx for idx, month in enumerate(calendar.month_name) if month}
    for name, num in months.items():
        if name in q:
            month_match = num
            break

    year_match_obj = re.search(r"\b(20\d{2})\b", q)
    if year_match_obj:
        year_match = int(year_match_obj.group(1))
    print(f"📅 Month match: {month_match}, Year match: {year_match}")

    # --- Apply filters ---
    filtered = filtered[filtered['sna_neighborhood'].str.contains(neighborhood_match, case=False, na=False)]

    generic_words = ["crime", "offense", "incident", "event", "report"]
    if offense_match and not any(word in q for word in generic_words):
        filtered = filtered[filtered['incident_type_desc'].str.contains(offense_match, case=False, na=False)]

    if month_match:
        filtered = filtered[filtered['create_time_incident'].dt.month == month_match]

    if year_match:
        filtered = filtered[filtered['create_time_incident'].dt.year == year_match]

    return filtered

# --- Summary generator ---
def generate_summary(question, filtered_df):
    if filtered_df.empty:
        return "No matching records found."

    q = question.lower()
    if "most common" in q and 'incident_type_desc' in filtered_df.columns:
        most_common = filtered_df['incident_type_desc'].value_counts().idxmax()
        count = filtered_df['incident_type_desc'].value_counts().max()
        return f"The most common crime is {most_common} with {count} incidents."

    examples = []
    for _, row in filtered_df.head(10).iterrows():
        try:
            date = row.get('create_time_incident', 'N/A')
            offense = row.get('incident_type_desc', 'N/A')
            neighborhood = row.get('sna_neighborhood', 'N/A')
            incident = row.get('event_number', 'N/A')
            examples.append(f"On {date}, a {offense} occurred in {neighborhood} (Incident #{incident}).")
        except:
            continue
    return "\n".join(examples)

# --- Main LLM handler ---
def answer_with_llm(question, data_rows, model, model_name="google/flan-t5-small", prompt_version="v3"):
    if data_rows.empty:
        return "Sorry, I couldn't find any data matching that question."

    q = question.lower()
    true_count = len(data_rows)

    if any(word in q for word in ["how many", "number of", "count"]):
        return f"There were {true_count} incidents that matched your query."

    context = generate_summary(question, data_rows.head(10))
    prompt = f"""
You are a helpful assistant analyzing Cincinnati crime data.

Here are some relevant data points:
{context}

Now answer this question based on the above:
{question}
    """.strip()

    start_time = time.time()
    with mlflow.start_run():
        mlflow.log_param("model_name", model_name)
        mlflow.log_param("max_new_tokens", 150)
        mlflow.log_param("num_rows_context", len(data_rows.head(10)))
        mlflow.log_param("prompt_version", prompt_version)
        mlflow.log_param("question", question)
        mlflow.log_param("true_count", true_count)
        mlflow.log_text(prompt, "prompt.txt")

        result = model(prompt, max_new_tokens=150)[0]['generated_text']
        answer = result.strip()
        latency = time.time() - start_time

        mlflow.log_text(answer, "answer.txt")
        mlflow.log_metric("response_length", len(answer))
        mlflow.log_metric("latency_sec", latency)

        return answer

# --- Run one Q&A turn ---
def run_single_chatbot_turn(df, model, question):
    if question.lower() in ['exit', 'quit']:
        return "Goodbye!"
    full_rows = get_relevant_rows(question, df)
    return answer_with_llm(question, full_rows, model)

# --- Widget interface ---
import ipywidgets as widgets
from IPython.display import display

def ask_question_with_widget(df, model):
    input_box = widgets.Text(
        placeholder='Ask a question about crime in Cincinnati...',
        description='Question:',
        layout=widgets.Layout(width='100%')
    )
    output = widgets.Output()

    def on_submit(change):
        output.clear_output()
        question = change['new']
        with output:
            print("Bot:", run_single_chatbot_turn(df, model, question))

    input_box.observe(on_submit, names='value')
    display(input_box, output)

# --- Launch the bot ---
print("Loading dataset and model...")
df = load_crime_data()
print("Columns in dataset:", df.columns.tolist())

llm_model = pipeline("text2text-generation", model="google/flan-t5-small", device=-1)
print("\n✅ Chatbot is ready! Ask your question below:")
ask_question_with_widget(df, llm_model)

Loading dataset and model...

✅ Sample Neighborhoods:
['MT. AIRY' 'WESTWOOD' 'WEST END' 'SOUTH FAIRMOUNT' 'CAMP WASHINGTON'
 'CORRYVILLE' 'EAST END' 'DOWNTOWN' 'AVONDALE' 'OVER-THE-RHINE']

📅 Sample Dates (Month-Year):
<PeriodArray>
['2021-11', '2021-10', '2021-09', '2021-08', '2021-07', '2021-06', '2021-05',
 '2021-04', '2020-10', '2021-03']
Length: 10, dtype: period[M]
Columns in dataset: ['address_x', 'agency', 'create_time_incident', 'disposition_text', 'event_number', 'incident_type_id', 'incident_type_desc', 'priority', 'priority_color', 'arrival_time_primary_unit', 'closed_time_incident', 'dispatch_time_primary_unit', 'beat', 'district', 'sna_neighborhood', 'cpd_neighborhood', 'community_council_neighborhood', 'latitude_x', 'longitude_x']


Device set to use cpu



✅ Chatbot is ready! Ask your question below:


Text(value='', description='Question:', layout=Layout(width='100%'), placeholder='Ask a question about crime i…

Output()